## 영문 pdf들 간 유사도 분석하기
정확한 문서간 유사도를 추출해냈다기보단<br>
이후 쓰일 ai 모델의 학습 지표로 참고할 만 하다~ 정도일듯

---
#### **TF-IDF(Term Frequency - Inverse Document Frequency)**


TF : 단어 빈도
<br>IDF : 단어가 나타난 문서 수(의 역수)


---



### 1. package installation



In [1]:
import csv
import os
from pathlib import Path
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import pandas as pd
from math import log
import datetime as dt

###2. txt to dataframe

In [4]:
#stopwords
nltk.download('stopwords')
print('checking stopwords . . .')
print(nltk.corpus.stopwords.words('english')[:20])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
checking stopwords . . .
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [5]:
path_dir = '../content/drive/MyDrive/dataset/wp_ether'
file_list = os.listdir(path_dir)
# file_list

In [6]:
# data : coin 이름과 whitepaper text를 가진 dataframe
#폴더에 있는 whitepaper들을 하나씩 읽어서 data에 저장함
data = pd.DataFrame()
for file in file_list:
  filename = file.replace('.txt', '')
  file = path_dir + '/' + file
  txt = Path(file).read_text()
  txt = txt.replace('\n', '')
  data=data.append({'coin' : filename , 'whitepaper' : txt} , ignore_index=True)
data

,coin,whitepaper
0,USD Coin,WhitepaperDocument Version 2.0 May 2018 3. Th...
1,Wrapped Bitcoin,Wrapped Tokens A multi-institutional framewor...
2,Aave,Protocol WhitepaperV1.0wow@aave.comJanuary 202...
3,Decentraland,White paper DecentralandA blockchain-based vir...
4,0x,0x: An open protocol for decentralized exchang...
...,...,...
576,Sylo,Sylo Protocol: Secure Group MessagingFelix Sch...
577,The Recharge,Companies constantly look for new means of pro...
578,Trias (old),"IntroductionThe IAS Machine, born in the 1950s..."
579,WeBlock,The Decentralized BlockchainAcceleration Solut...


In [7]:
data_pr = data
data_pr['whitepaper']

0      WhitepaperDocument Version 2.0  May 2018
3. Th...
1       Wrapped Tokens A multi-institutional framewor...
2      Protocol WhitepaperV1.0wow@aave.comJanuary 202...
3      White paper
DecentralandA blockchain-based vir...
4      0x: An open protocol for decentralized exchang...
                             ...                        
576    Sylo Protocol: Secure Group MessagingFelix Sch...
577    Companies constantly look for new means of pro...
578    IntroductionThe IAS Machine, born in the 1950s...
579    The Decentralized BlockchainAcceleration Solut...
580    XFai and Our DLOVersion 1.0 DraftXFai is a set...
Name: whitepaper, Length: 581, dtype: object

###3. text preprocessing

In [8]:
stop_word = set(stopwords.words('english'))

In [9]:
#특수문자 없앰
data_pr['whitepaper'] = data_pr['whitepaper'].str.replace("[^a-zA-Z\' ]","")
data_pr['whitepaper']

0      WhitepaperDocument Version   May  The CENTRE T...
1      Wrapped TokensA multiinstitutional framework f...
2      Protocol WhitepaperVwowaavecomJanuary Abstract...
3      White paperDecentralandA blockchainbased virtu...
4      x An open protocol for decentralized exchange ...
                             ...                        
576    Sylo Protocol Secure Group MessagingFelix Schl...
577    Companies constantly look for new means of pro...
578    IntroductionThe IAS Machine born in the s repr...
579    The Decentralized BlockchainAcceleration Solut...
580    XFai and Our DLOVersion  DraftXFai is a set of...
Name: whitepaper, Length: 581, dtype: object

In [10]:
#영어 소문자로 통일
data_pr['whitepaper'] = data_pr['whitepaper'].apply(lambda x: ' '.join([word.lower() for word in x.split() if word not in (stop_word)]))

In [11]:
data_pr['whitepaper']

0      whitepaperdocument version may the centre team...
1      wrapped tokensa multiinstitutional framework t...
2      protocol whitepapervwowaavecomjanuary abstract...
3      white paperdecentralanda blockchainbased virtu...
4      x an open protocol decentralized exchange onth...
                             ...                        
576    sylo protocol secure group messagingfelix schl...
577    companies constantly look new means production...
578    introductionthe ias machine born represents ne...
579    the decentralized blockchainacceleration solut...
580    xfai our dloversion draftxfai set autonomous l...
Name: whitepaper, Length: 581, dtype: object

###4. calculating similarites

In [12]:
#TfidfVectorizer 사용했음
#2번이하, 20번 이상 나오는 단어들은 제외
#ngram range는 (2, 3)

v = TfidfVectorizer(max_df = 20, min_df=2, ngram_range=(2,3))
x = v.fit_transform(data_pr['whitepaper'])

In [ ]:
v.vocabulary_

In [14]:
similarities = (x * x.T)

In [15]:
#편리한 계산을 위해 자기자신과의 유사도(=1)을 0으로 바꿔줌

result = similarities.toarray()
result = np.where(result>=0.99, 0, result)
result

array([[0.        , 0.01070632, 0.00082104, ..., 0.00503407, 0.0072997 ,
        0.00430324],
       [0.01070632, 0.        , 0.        , ..., 0.0024114 , 0.00311973,
        0.00757277],
       [0.00082104, 0.        , 0.        , ..., 0.        , 0.00098314,
        0.00403202],
       ...,
       [0.00503407, 0.0024114 , 0.        , ..., 0.        , 0.00513992,
        0.00217633],
       [0.0072997 , 0.00311973, 0.00098314, ..., 0.00513992, 0.        ,
        0.003259  ],
       [0.00430324, 0.00757277, 0.00403202, ..., 0.00217633, 0.003259  ,
        0.        ]])

In [16]:
result = pd.DataFrame(result)
idx_copy_max = result.idxmax(axis=1)
result

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580
0,0.000000,0.010706,0.000821,0.017552,0.033505,0.003566,0.003566,0.006530,0.007686,0.016028,0.001633,0.006941,0.004026,0.020685,0.003732,0.003587,0.005420,0.016221,0.010062,0.002184,0.006052,0.002699,0.001868,0.003781,0.010015,0.012771,0.003545,0.006414,0.006477,0.004261,0.007190,0.002184,0.005099,0.008935,0.002774,0.001263,0.008490,0.010960,0.008224,0.006443,...,0.000000,0.004981,0.004231,0.000495,0.018929,0.000635,0.000000,0.013403,0.000000,0.000000,0.008191,0.005655,0.0,0.001262,0.001462,0.0,0.013714,0.002501,0.000000,0.000000,0.001079,0.000000,0.008640,0.001850,0.005009,0.006681,0.007356,0.005187,0.001845,0.003498,0.001409,0.001409,0.004436,0.004671,0.005006,0.014656,0.002576,0.005034,0.007300,0.004303
1,0.010706,0.000000,0.000000,0.003429,0.008576,0.005785,0.005785,0.005252,0.002791,0.002469,0.000000,0.002089,0.005864,0.004700,0.003266,0.001544,0.004390,0.008664,0.005896,0.002423,0.001712,0.002209,0.007258,0.003519,0.003752,0.015014,0.002240,0.004574,0.002626,0.011090,0.005199,0.002423,0.001507,0.006069,0.002947,0.002998,0.003588,0.000741,0.003607,0.002803,...,0.000000,0.001359,0.001849,0.006237,0.015859,0.002935,0.000000,0.011544,0.000000,0.000000,0.004338,0.002112,0.0,0.000796,0.003139,0.0,0.012100,0.000000,0.003855,0.002919,0.004081,0.000000,0.000830,0.001192,0.001333,0.001266,0.003425,0.005102,0.001437,0.002250,0.003411,0.003411,0.001401,0.002900,0.005030,0.001896,0.000000,0.002411,0.003120,0.007573
2,0.000821,0.000000,0.000000,0.001385,0.001115,0.000384,0.000384,0.000219,0.003648,0.000444,0.000000,0.000179,0.002178,0.001665,0.001023,0.000000,0.000711,0.003117,0.000259,0.000720,0.000000,0.000615,0.001481,0.001470,0.000560,0.002688,0.007375,0.000192,0.000261,0.002555,0.001164,0.000720,0.000412,0.000284,0.000682,0.002690,0.001431,0.000683,0.001991,0.001027,...,0.000000,0.000321,0.000155,0.000427,0.003728,0.000000,0.000000,0.002807,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.002166,0.0,0.001441,0.000419,0.000000,0.000000,0.001792,0.000000,0.000599,0.000000,0.000262,0.000000,0.000284,0.000801,0.000379,0.000000,0.000971,0.000971,0.002120,0.000345,0.000486,0.002141,0.000000,0.000000,0.000983,0.004032
3,0.017552,0.003429,0.001385,0.000000,0.016481,0.003653,0.003653,0.008049,0.000711,0.005859,0.000000,0.009442,0.008663,0.013201,0.004373,0.004111,0.012333,0.021340,0.019931,0.002274,0.001541,0.009783,0.001718,0.005642,0.010700,0.005341,0.005069,0.003775,0.000514,0.006105,0.002971,0.002274,0.003632,0.025426,0.008148,0.003960,0.007541,0.005245,0.005595,0.002019,...,0.000000,0.003290,0.005474,0.007077,0.008011,0.001884,0.004608,0.004796,0.001153,0.003002,0.008551,0.004910,0.0,0.003018,0.000000,0.0,0.009916,0.001220,0.000000,0.002122,0.031094,0.000000,0.001019,0.000000,0.002207,0.001158,0.004167,0.002336,0.001529,0.003794,0.000477,0.000477,0.001020,0.002553,0.005204,0.005458,0.002314,0.001977,0.004876,0.001311
4,0.033505,0.008576,0.001115,0.016481,0.000000,0.004104,0.004104,0.006112,0.015084,0.001019,0.000000,0.004494,0.004146,0.005164,0.006824,0.009852,0.015437,0.007701,0.003524,0.001847,0.001055,0.001641,0.002697,0.004838,0.000790,0.032669,0.011871,0.004282,0.003514,0.011568,0.001844,0.001847,0.000511,0.010934,0.005686,0.003250,0.012385,0.017411,0.020061,0.004026,...,0.000000,0.001927,0.004803,0.008408,0.008266,0.000000,0.000000,0.009670,0.002778,0.002053,0.000794,0.000000,0.0,0.000000,0.000861,0.0,0.011878,0.005737,0.000000,0.000000,0.006624,0.000000,0.000000,0.001542,0.001271,0.003233,0.006593,0.004977,0.002333,0.001865,0.003533,0.003533,0.001929,0.001514,0.002177,0.008531,0.000000,0.001431,0.004046,0.011563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [17]:
#최종 결과 csv로 만들 애들을 feature dataframe으로 모았음
#feature column들은 밑에서 설명

feature = pd.DataFrame(data = data_pr['coin'])  

In [18]:
feature['copy_max'] = result.max(axis=1, numeric_only=True)
feature['idx_copy_max'] = idx_copy_max
feature['id'] = feature.index
feature

,coin,copy_max,idx_copy_max,id
0,USD Coin,0.072171,171,0
1,Wrapped Bitcoin,0.076113,262,1
2,Aave,0.075866,276,2
3,Decentraland,0.080415,450,3
4,0x,0.034892,504,4
...,...,...,...,...
576,Sylo,0.029831,140,576
577,The Recharge,0.081203,69,577
578,Trias (old),0.036558,245,578
579,WeBlock,0.041981,232,579


In [19]:
coin_name = feature['coin']
feature['coin_copy_max'] = feature['idx_copy_max'].apply(lambda x : coin_name[x])

In [20]:
cnt = result.apply(lambda x : x.value_counts())

In [21]:
cnt = cnt.iloc[cnt.index>=0.15]
cnt = cnt.fillna(0)

In [22]:
cnt_copy = cnt.sum(axis = 0)

In [23]:
feature['cnt_copy'] = cnt_copy

In [24]:
feature.sort_values(by=['copy_max'], ascending=False)

,coin,copy_max,idx_copy_max,id,coin_copy_max,cnt_copy
254,Parallel,0.988476,535,254,Parallel Protocol,1.0
535,Parallel Protocol,0.988476,254,535,Parallel,1.0
277,IOU,0.910662,188,277,VAIOT,2.0
188,VAIOT,0.910662,277,188,IOU,2.0
374,BQT,0.780874,277,374,IOU,2.0
...,...,...,...,...,...,...
421,Mirrored Amazon,0.004829,26,421,Strike,0.0
419,Mirrored Invesco QQQ Trust,0.004829,26,419,Strike,0.0
418,Mirror Protocol,0.004829,26,418,Strike,0.0
420,Mirrored Microsoft,0.004829,26,420,Strike,0.0


###5. postprocessing
표절했는지, 당했는지 발행날짜 참고하여 표절률 구분

####5.1 코인 발행날짜 받아오기

In [25]:
path = '../content/drive/MyDrive/dataset/coin_addedDate.csv'

In [26]:
data = pd.read_csv(path)
data.head()

,Unnamed: 0,id,name,date_added
0,0,1,Bitcoin,2013-04-28T00:00:00.000Z
1,1,1027,Ethereum,2015-08-07T00:00:00.000Z
2,2,2010,Cardano,2017-10-01T00:00:00.000Z
3,3,1839,Binance Coin,2017-07-25T00:00:00.000Z
4,4,825,Tether,2015-02-25T00:00:00.000Z


In [27]:
data = data.drop(columns = ['id'])
data.columns = ['id',	'name',	'date_added']
data

,id,name,date_added
0,0,Bitcoin,2013-04-28T00:00:00.000Z
1,1,Ethereum,2015-08-07T00:00:00.000Z
2,2,Cardano,2017-10-01T00:00:00.000Z
3,3,Binance Coin,2017-07-25T00:00:00.000Z
4,4,Tether,2015-02-25T00:00:00.000Z
...,...,...,...
6515,6515,InfinityCake,2021-09-06T16:33:27.000Z
6516,6516,Foobee,2021-09-06T16:44:23.000Z
6517,6517,Fantasy Tales,2021-09-06T17:52:10.000Z
6518,6518,Wrapped One,2021-09-06T18:15:45.000Z


In [28]:
def str_to_date(x) :
  if str(x).find("T") != -1 :
    delete = str(x).split("T")[1]  #헤지니이ㅣ
    x = x.replace(delete, "").replace(delete, "")
  return x

In [29]:
data['date_added'] = data['date_added'].apply(str_to_date).astype('datetime64[D]')
coin_name_list = list(coin_name)

In [30]:
coin_date_added = data['name'].apply(lambda x : 1 if x in coin_name_list else 0)
coin_date_added = pd.DataFrame(data = coin_date_added)
coin_date_added.columns = ['deleteornot']
data1 = pd.concat([data, coin_date_added], axis =1)
data1 = data1.drop(columns = ['id'])

In [31]:
idx = data1[data1['deleteornot']==0].index
data_res = data1.drop(idx)

In [32]:
concat_f = feature
concat_f.columns = ['coinname', 'copy_max', 'idx_copy_max', 'id', 'coin_copy_max', 'cnt_copy']
concat_f = concat_f.drop_duplicates(['coinname'])

concat_d = data_res
concat_d.columns = ['coinname', 'date_added', 'deleteornot']
concat_d = concat_d.drop_duplicates(['coinname'])

In [33]:
data_concat = pd.merge(concat_f, concat_d, on='coinname', how = 'left')
data_concat

,coinname,copy_max,idx_copy_max,id,coin_copy_max,cnt_copy,date_added,deleteornot
0,USD Coin,0.072171,171,0,Privatix,0.0,2018-10-08,1.0
1,Wrapped Bitcoin,0.076113,262,1,Pantos,0.0,2019-01-30,1.0
2,Aave,0.075866,276,2,Impermax,0.0,2020-10-02,1.0
3,Decentraland,0.080415,450,3,Meter Governance mapped by Meter.io,0.0,2017-09-17,1.0
4,0x,0.034892,504,4,Rai Reflex Index,0.0,2017-08-16,1.0
...,...,...,...,...,...,...,...,...
576,Sylo,0.029831,140,576,4THPILLAR TECHNOLOGIES,0.0,2020-06-05,1.0
577,The Recharge,0.081203,69,577,Electric Vehicle Zone,0.0,2021-09-02,1.0
578,Trias (old),0.036558,245,578,pTokens BTC,0.0,2019-05-24,1.0
579,WeBlock,0.041981,232,579,AICON,0.0,2020-10-26,1.0


In [34]:
data_concat.sort_values(by=['copy_max'], ascending=False)

,coinname,copy_max,idx_copy_max,id,coin_copy_max,cnt_copy,date_added,deleteornot
254,Parallel,0.988476,535,254,Parallel Protocol,1.0,2021-03-03,1.0
535,Parallel Protocol,0.988476,254,535,Parallel,1.0,2021-07-11,1.0
277,IOU,0.910662,188,277,VAIOT,2.0,2019-06-05,1.0
188,VAIOT,0.910662,277,188,IOU,2.0,2021-04-20,1.0
374,BQT,0.780874,277,374,IOU,2.0,2019-05-10,1.0
...,...,...,...,...,...,...,...,...
421,Mirrored Amazon,0.004829,26,421,Strike,0.0,2020-12-16,1.0
419,Mirrored Invesco QQQ Trust,0.004829,26,419,Strike,0.0,2020-12-16,1.0
418,Mirror Protocol,0.004829,26,418,Strike,0.0,2020-12-04,1.0
420,Mirrored Microsoft,0.004829,26,420,Strike,0.0,2020-12-16,1.0


In [35]:
data_concat = data_concat.drop(columns=['coin_copy_max', 'deleteornot']).dropna(how='any')
data_concat = data_concat.drop_duplicates(["coinname"])
data_concat

,coinname,copy_max,idx_copy_max,id,cnt_copy,date_added
0,USD Coin,0.072171,171,0,0.0,2018-10-08
1,Wrapped Bitcoin,0.076113,262,1,0.0,2019-01-30
2,Aave,0.075866,276,2,0.0,2020-10-02
3,Decentraland,0.080415,450,3,0.0,2017-09-17
4,0x,0.034892,504,4,0.0,2017-08-16
...,...,...,...,...,...,...
576,Sylo,0.029831,140,576,0.0,2020-06-05
577,The Recharge,0.081203,69,577,0.0,2021-09-02
578,Trias (old),0.036558,245,578,0.0,2019-05-24
579,WeBlock,0.041981,232,579,0.0,2020-10-26


####5.2 발행날짜 고려해서 표절당한애들 feature 보정하기

In [36]:
result_to_correct = result  #바뀔애들이 사용할 similarity matrix

In [37]:
to_correct = pd.DataFrame()  #표절 아니라고 나온 애들 모아둔거

def correcting_result(df) : #data_concat을 받아와서 한 행씩 작업 할거임
  global to_correct
  global result_to_correct
  global to_correct

  to_correct = pd.DataFrame()
  result_to_correct = result  #바뀔애들이 사용할 similarity matrix

  for i in df.index : 
    print(i)
    x = df.loc[i]    
    print(x)
    print(result_to_correct[i][x['idx_copy_max']])
    if x['copy_max'] <= 0.15 :
      continue
    coin = x['coinname']
    copied = df.loc[x['idx_copy_max']]
    rate = x['copy_max']
    copied_or_not = (x['date_added'] - copied['date_added']).days > 0  #x가 copy_max 코인 보다 늦게 발행되었다면 true
    if copied_or_not :  #표절 한거면
      print(coin, ' has coied', copied['coinname'], '   percent : ', rate)
    else :
      print(coin, 'has not copied', copied['coinname'], '   percent : ', rate)
      to_correct = to_correct.append(x)
      rate_result = result_to_correct[i][x['idx_copy_max']]
      print(rate_result)
      result_to_correct[i][x['idx_copy_max']] = 0
      result_to_correct[x['idx_copy_max']][i] = 0

In [38]:
result_to_correct

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580
0,0.000000,0.010706,0.000821,0.017552,0.033505,0.003566,0.003566,0.006530,0.007686,0.016028,0.001633,0.006941,0.004026,0.020685,0.003732,0.003587,0.005420,0.016221,0.010062,0.002184,0.006052,0.002699,0.001868,0.003781,0.010015,0.012771,0.003545,0.006414,0.006477,0.004261,0.007190,0.002184,0.005099,0.008935,0.002774,0.001263,0.008490,0.010960,0.008224,0.006443,...,0.000000,0.004981,0.004231,0.000495,0.018929,0.000635,0.000000,0.013403,0.000000,0.000000,0.008191,0.005655,0.0,0.001262,0.001462,0.0,0.013714,0.002501,0.000000,0.000000,0.001079,0.000000,0.008640,0.001850,0.005009,0.006681,0.007356,0.005187,0.001845,0.003498,0.001409,0.001409,0.004436,0.004671,0.005006,0.014656,0.002576,0.005034,0.007300,0.004303
1,0.010706,0.000000,0.000000,0.003429,0.008576,0.005785,0.005785,0.005252,0.002791,0.002469,0.000000,0.002089,0.005864,0.004700,0.003266,0.001544,0.004390,0.008664,0.005896,0.002423,0.001712,0.002209,0.007258,0.003519,0.003752,0.015014,0.002240,0.004574,0.002626,0.011090,0.005199,0.002423,0.001507,0.006069,0.002947,0.002998,0.003588,0.000741,0.003607,0.002803,...,0.000000,0.001359,0.001849,0.006237,0.015859,0.002935,0.000000,0.011544,0.000000,0.000000,0.004338,0.002112,0.0,0.000796,0.003139,0.0,0.012100,0.000000,0.003855,0.002919,0.004081,0.000000,0.000830,0.001192,0.001333,0.001266,0.003425,0.005102,0.001437,0.002250,0.003411,0.003411,0.001401,0.002900,0.005030,0.001896,0.000000,0.002411,0.003120,0.007573
2,0.000821,0.000000,0.000000,0.001385,0.001115,0.000384,0.000384,0.000219,0.003648,0.000444,0.000000,0.000179,0.002178,0.001665,0.001023,0.000000,0.000711,0.003117,0.000259,0.000720,0.000000,0.000615,0.001481,0.001470,0.000560,0.002688,0.007375,0.000192,0.000261,0.002555,0.001164,0.000720,0.000412,0.000284,0.000682,0.002690,0.001431,0.000683,0.001991,0.001027,...,0.000000,0.000321,0.000155,0.000427,0.003728,0.000000,0.000000,0.002807,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.002166,0.0,0.001441,0.000419,0.000000,0.000000,0.001792,0.000000,0.000599,0.000000,0.000262,0.000000,0.000284,0.000801,0.000379,0.000000,0.000971,0.000971,0.002120,0.000345,0.000486,0.002141,0.000000,0.000000,0.000983,0.004032
3,0.017552,0.003429,0.001385,0.000000,0.016481,0.003653,0.003653,0.008049,0.000711,0.005859,0.000000,0.009442,0.008663,0.013201,0.004373,0.004111,0.012333,0.021340,0.019931,0.002274,0.001541,0.009783,0.001718,0.005642,0.010700,0.005341,0.005069,0.003775,0.000514,0.006105,0.002971,0.002274,0.003632,0.025426,0.008148,0.003960,0.007541,0.005245,0.005595,0.002019,...,0.000000,0.003290,0.005474,0.007077,0.008011,0.001884,0.004608,0.004796,0.001153,0.003002,0.008551,0.004910,0.0,0.003018,0.000000,0.0,0.009916,0.001220,0.000000,0.002122,0.031094,0.000000,0.001019,0.000000,0.002207,0.001158,0.004167,0.002336,0.001529,0.003794,0.000477,0.000477,0.001020,0.002553,0.005204,0.005458,0.002314,0.001977,0.004876,0.001311
4,0.033505,0.008576,0.001115,0.016481,0.000000,0.004104,0.004104,0.006112,0.015084,0.001019,0.000000,0.004494,0.004146,0.005164,0.006824,0.009852,0.015437,0.007701,0.003524,0.001847,0.001055,0.001641,0.002697,0.004838,0.000790,0.032669,0.011871,0.004282,0.003514,0.011568,0.001844,0.001847,0.000511,0.010934,0.005686,0.003250,0.012385,0.017411,0.020061,0.004026,...,0.000000,0.001927,0.004803,0.008408,0.008266,0.000000,0.000000,0.009670,0.002778,0.002053,0.000794,0.000000,0.0,0.000000,0.000861,0.0,0.011878,0.005737,0.000000,0.000000,0.006624,0.000000,0.000000,0.001542,0.001271,0.003233,0.006593,0.004977,0.002333,0.001865,0.003533,0.003533,0.001929,0.001514,0.002177,0.008531,0.000000,0.001431,0.004046,0.011563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [39]:
correcting_result(data_concat)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0.06605398794830868
42
coinname                    Valobit
copy_max                  0.0258722
idx_copy_max                    490
id                               42
cnt_copy                          0
date_added      2020-09-08 00:00:00
Name: 42, dtype: object
0.025872169794146575
43
coinname                       REVV
copy_max                   0.314111
idx_copy_max                     40
id                               43
cnt_copy                          1
date_added      2020-09-10 00:00:00
Name: 43, dtype: object
0.0
REVV  has coied Presearch    percent :  0.31411069921699514
44
coinname             GoMining token
copy_max                  0.0274923
idx_copy_max                    199
id                               44
cnt_copy                          0
date_added      2021-06-01 00:00:00
Name: 44, dtype: object
0.02749227261574779
45
coinname                 Ultiledger
copy_max                   0.105854
idx_copy_max                    470

In [40]:
to_correct

,cnt_copy,coinname,copy_max,date_added,id,idx_copy_max
15,1.0,StormX,0.310462,2017-12-20,15.0,172.0
20,1.0,The Transfer Token,0.428605,2020-04-29,20.0,573.0
22,2.0,PEAKDEFI,0.175151,2020-03-23,22.0,565.0
36,1.0,Streamr,0.165737,2017-11-03,36.0,294.0
40,1.0,Presearch,0.314111,2017-12-05,40.0,43.0
100,1.0,PolySwarm,0.279136,2018-04-10,100.0,77.0
105,1.0,MahaDAO,0.160065,2020-12-17,105.0,454.0
108,1.0,Graphlinq Protocol,0.314355,2021-03-31,108.0,334.0
109,2.0,U Network,0.212987,2018-04-17,109.0,569.0
132,1.0,TrueFlip,0.319138,2017-08-23,132.0,311.0


In [41]:
result_to_correct = pd.DataFrame(result_to_correct)
idx_copy_max2 = result_to_correct.idxmax(axis = 1)
feature1 = pd.DataFrame()
feature1['coinname'] = feature['coinname']
feature1['copy_max'] = result_to_correct.max(axis=1, numeric_only=True)
feature1['idx_copy_max'] = idx_copy_max2
feature1['id'] = feature.index
feature1['coin_copy_max'] = feature1['idx_copy_max'].apply(lambda x : coin_name[x])

cnt2 = result_to_correct.apply(lambda x : x.value_counts())
cnt2 = cnt2.iloc[cnt2.index>=0.15]
cnt2 = cnt2.fillna(0)

cnt_copy2 = cnt2.sum(axis = 0)
feature1['cnt_copy'] = cnt_copy2
feature1

,coinname,copy_max,idx_copy_max,id,coin_copy_max,cnt_copy
0,USD Coin,0.072171,171,0,Privatix,0.0
1,Wrapped Bitcoin,0.076113,262,1,Pantos,0.0
2,Aave,0.075866,276,2,Impermax,0.0
3,Decentraland,0.080415,450,3,Meter Governance mapped by Meter.io,0.0
4,0x,0.034892,504,4,Rai Reflex Index,0.0
...,...,...,...,...,...,...
576,Sylo,0.029831,140,576,4THPILLAR TECHNOLOGIES,0.0
577,The Recharge,0.081203,69,577,Electric Vehicle Zone,0.0
578,Trias (old),0.036558,245,578,pTokens BTC,0.0
579,WeBlock,0.041981,232,579,AICON,0.0


In [42]:
feature1.sort_values(by=['copy_max'], ascending=False)

,coinname,copy_max,idx_copy_max,id,coin_copy_max,cnt_copy
374,BQT,0.750177,188,374,VAIOT,1.0
188,VAIOT,0.750177,374,188,BQT,1.0
541,Orient,0.376670,380,541,CURE Farm,6.0
380,CURE Farm,0.376670,541,380,Orient,3.0
546,QuickSwap,0.301961,541,546,Orient,3.0
...,...,...,...,...,...,...
422,Mirrored iShares Silver Trust,0.004829,26,422,Strike,0.0
423,Mirrored Twitter,0.004829,26,423,Strike,0.0
424,Mirrored United States Oil Fund,0.004829,26,424,Strike,0.0
256,VROOMGO,0.004124,47,256,ForTube,0.0


In [45]:
coin_to_correct_list = to_correct['id']
list(coin_to_correct_list)
coin_to_correct_list = coin_to_correct_list.astype('int')

In [46]:
feature_corrected_1 = feature
for i in feature1.index : 
  if i in coin_to_correct_list:
    print(i, feature.loc[i, 'coinname'])
    if feature.loc[i, 'coinname'] != feature1.loc[i, 'coinname'] :
      print('error')
      break;
    print('before : ')
    print(feature.loc[i])
    feature.loc[i] = feature1.loc[i]
    print('after : ')
    print(feature.loc[i])
    print('----------')

15 StormX
before : 
coinname            StormX
copy_max          0.310462
idx_copy_max           172
id                      15
coin_copy_max    Rewardiqa
cnt_copy                 1
Name: 15, dtype: object
after : 
coinname            StormX
copy_max         0.0779088
idx_copy_max           139
id                      15
coin_copy_max       Bancor
cnt_copy                 0
Name: 15, dtype: object
----------
20 The Transfer Token
before : 
coinname         The Transfer Token
copy_max                   0.428605
idx_copy_max                    573
id                               20
coin_copy_max            Sensitrust
cnt_copy                          1
Name: 20, dtype: object
after : 
coinname         The Transfer Token
copy_max                   0.107964
idx_copy_max                    525
id                               20
coin_copy_max                 Libfx
cnt_copy                          0
Name: 20, dtype: object
----------
22 PEAKDEFI
before : 
coinname             PEAKDEFI
copy

,coinname,copy_max,idx_copy_max,id,coin_copy_max,cnt_copy
541,Orient,0.376670,380,541,CURE Farm,6.0
376,BT.Finance,0.376844,541,376,Orient,4.0
380,CURE Farm,0.376670,541,380,Orient,3.0
251,BabyDoge ETH,0.293405,242,251,Kelpie Inu,3.0
333,EmploymentCoin,0.519296,313,333,ARMTOKEN,3.0
...,...,...,...,...,...,...
194,Hokkaidu Inu,0.138866,234,194,KleeKai,0.0
193,Flurry Finance,0.050213,231,193,EnreachDAO,0.0
192,PROXI,0.041218,433,192,PERL.eco,0.0
191,Pando,0.142069,474,191,Insight Protocol,0.0


In [47]:
feature_corrected_1.sort_values(by=['copy_max'], ascending=False)

,coinname,copy_max,idx_copy_max,id,coin_copy_max,cnt_copy
535,Parallel Protocol,0.988476,254,535,Parallel,1.0
188,VAIOT,0.910662,277,188,IOU,2.0
374,BQT,0.750177,188,374,VAIOT,1.0
163,Innovative Bioresearch Coin,0.651987,170,163,Transcodium,2.0
482,Ethereum Pro OLD CONTRACT,0.540136,462,482,Benchmark Protocol,1.0
...,...,...,...,...,...,...
418,Mirror Protocol,0.004829,26,418,Strike,0.0
416,Mirrored Facebook Inc,0.004829,26,416,Strike,0.0
417,Mirrored ProShares VIX,0.004829,26,417,Strike,0.0
256,VROOMGO,0.004124,47,256,ForTube,0.0


In [48]:
final_result = feature_corrected_1

###6. results


---
feature 설명<br>
* coin : 코인 이름<br>
* coin_max : 가장 높게 나온 유사도 수치<br>
* idx_copy_max : white paper 유사도가 가장 높게 나온 코인의 index<br>
* coin_copy_max : white paper 유사도가 가장 높게 나온 코인의 이름<br>
* cnt_copy : white paper 유사도가 15%이상 나온 문서의 개수<br>


---




In [49]:
final_result.sort_values(by=['cnt_copy'], ascending=False)

,coinname,copy_max,idx_copy_max,id,coin_copy_max,cnt_copy
541,Orient,0.376670,380,541,CURE Farm,6.0
376,BT.Finance,0.376844,541,376,Orient,4.0
380,CURE Farm,0.376670,541,380,Orient,3.0
251,BabyDoge ETH,0.293405,242,251,Kelpie Inu,3.0
333,EmploymentCoin,0.519296,313,333,ARMTOKEN,3.0
...,...,...,...,...,...,...
194,Hokkaidu Inu,0.138866,234,194,KleeKai,0.0
193,Flurry Finance,0.050213,231,193,EnreachDAO,0.0
192,PROXI,0.041218,433,192,PERL.eco,0.0
191,Pando,0.142069,474,191,Insight Protocol,0.0


In [50]:
final_result.sort_values(by=['copy_max'], ascending=False)

,coinname,copy_max,idx_copy_max,id,coin_copy_max,cnt_copy
535,Parallel Protocol,0.988476,254,535,Parallel,1.0
188,VAIOT,0.910662,277,188,IOU,2.0
374,BQT,0.750177,188,374,VAIOT,1.0
163,Innovative Bioresearch Coin,0.651987,170,163,Transcodium,2.0
482,Ethereum Pro OLD CONTRACT,0.540136,462,482,Benchmark Protocol,1.0
...,...,...,...,...,...,...
418,Mirror Protocol,0.004829,26,418,Strike,0.0
416,Mirrored Facebook Inc,0.004829,26,416,Strike,0.0
417,Mirrored ProShares VIX,0.004829,26,417,Strike,0.0
256,VROOMGO,0.004124,47,256,ForTube,0.0


In [53]:
feature.to_csv('feature_wp.csv', sep=',')